# IEEE Fraud detection adversarial validation [lightgbm]
* IEEE Fraud detection train/test data binary classification task.
* Reference notebook: <https://www.kaggle.com/code/jtrotman/ieee-fraud-adversarial-lgb-split-points/notebook>
* Dataset: <https://www.kaggle.com/code/jtrotman/ieee-fraud-adversarial-lgb-split-points/input>

By running this notebook, you’ll create a whole test suite in a few lines of code. The model used here is a LGBM classification model with the IEEE fraud detection dataset. Feel free to use your own model (tabular, text, or LLM).

You’ll learn how to:
* Detect vulnerabilities by scanning the model
* Generate a test suite with domain-specific tests
* Customize your test suite by loading a test from the Giskard catalog
* Upload your model to the Giskard server to:
    * Compare models to decide which one to promote
    * Debug your tests to diagnose issues
    * Share your results and collect business feedback from your team

## Install Giskard

In [ ]:
!pip install giskard

## Import libraries

In [2]:
import os
from pathlib import Path
from urllib.request import urlretrieve

import numpy as np
import pandas as pd
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from pandas.api.types import union_categoricals
from sklearn.model_selection import train_test_split

import giskard
from giskard import GiskardClient, testing, Dataset, Model

## Define constants

In [3]:
# Constants.
TARGET_COLUMN = 'isTest'
IDX_LABEL = 'TransactionID'

# Paths.
DATA_URL = os.path.join("ftp://sys.giskard.ai", "pub", "unit_test_resources", "fraud_detection_classification_dataset", "{}")
DATA_PATH = Path.home() / ".giskard" / "fraud_detection_classification_dataset"

## Dataset preparation

###  Load and preprocess data

In [4]:
def fetch_from_ftp(url: str, file: Path) -> None:
    """Helper to fetch data from the FTP server."""
    if not file.parent.exists():
        file.parent.mkdir(parents=True, exist_ok=True)

    if not file.exists():
        print(f"Downloading data from {url}")
        urlretrieve(url, file)

    print(f"Data was loaded!")


def fetch_dataset():
    files_to_fetch = ["train_transaction.csv", "train_identity.csv", "test_transaction.csv", "test_identity.csv"]
    for file_name in files_to_fetch:
        fetch_from_ftp(DATA_URL.format(file_name), DATA_PATH / file_name)


# Define data-types of transactions features.
DATA_TYPES_TRANSACTION = {
    'TransactionID': 'int32',
    'isFraud': 'int8',
    'TransactionDT': 'int32',
    'TransactionAmt': 'float32',
    'ProductCD': 'category',
    'card1': 'int16',
    'card2': 'float32',
    'card3': 'float32',
    'card4': 'category',
    'card5': 'float32',
    'card6': 'category',
    'addr1': 'float32',
    'addr2': 'float32',
    'dist1': 'float32',
    'dist2': 'float32',
    'P_emaildomain': 'category',
    'R_emaildomain': 'category',
}

C_COLS = [f'C{i}' for i in range(1, 15)]
D_COLS = [f'D{i}' for i in range(1, 16)]
M_COLS = [f'M{i}' for i in range(1, 10)]
V_COLS = [f'V{i}' for i in range(1, 340)]

DATA_TYPES_TRANSACTION.update((c, 'float32') for c in C_COLS)
DATA_TYPES_TRANSACTION.update((c, 'float32') for c in D_COLS)
DATA_TYPES_TRANSACTION.update((c, 'float32') for c in V_COLS)
DATA_TYPES_TRANSACTION.update((c, 'category') for c in M_COLS)

# Define datatypes of identity features.
DATA_TYPES_ID = {
    'TransactionID': 'int32',
    'DeviceType': 'category',
    'DeviceInfo': 'category',
}

ID_COLS = [f'id_{i:02d}' for i in range(1, 39)]
ID_CATS = [
    'id_12', 'id_15', 'id_16', 'id_23', 'id_27', 'id_28', 'id_29', 'id_30',
    'id_31', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38'
]

DATA_TYPES_ID.update(((c, 'float32') for c in ID_COLS))
DATA_TYPES_ID.update(((c, 'category') for c in ID_CATS))

# Define list of all categorical features.
CATEGORICALS = [f_name for (f_name, f_type) in dict(DATA_TYPES_TRANSACTION, **DATA_TYPES_ID).items() if f_type == "category"]


def read_set(_type):
    """Read both transactions and identity data."""
    print(f"Reading transactions data...")
    _df = pd.read_csv(os.path.join(DATA_PATH, f'{_type}_transaction.csv'),
                      index_col=IDX_LABEL, dtype=DATA_TYPES_TRANSACTION, nrows=250)

    print(f"Reading identity data...")
    _df = _df.join(pd.read_csv(os.path.join(DATA_PATH, f'{_type}_identity.csv'),
                               index_col=IDX_LABEL, dtype=DATA_TYPES_ID))
    return _df

def read_dataset():
    """Read whole data."""
    fetch_dataset()

    print(f"Reading train data...")
    train_set = read_set('train')

    print(f"Reading test data...")
    test_set = read_set('test')

    return train_set, test_set

In [5]:
def preprocess_dataset(train_set, test_set):
    """Unite train and test into common dataframe."""
    # Create a new target column and remove a former one from the train data.
    print("Start data preprocessing...")
    train_set.pop('isFraud')
    train_set['isTest'] = 0
    test_set['isTest'] = 1

    # Preprocess categorical features.
    n_train = train_set.shape[0]
    for c in train_set.columns:
        s = train_set[c]
        if hasattr(s, 'cat'):
            u = union_categoricals([train_set[c], test_set[c]], sort_categories=True)
            train_set[c] = u[:n_train]
            test_set[c] = u[n_train:]

    # Unite train and test data.
    united = pd.concat([train_set, test_set])

    # Add additional features.
    united['TimeInDay'] = united.TransactionDT % 86400
    united['Cents'] = united.TransactionAmt % 1

    # Remove useless columns.
    united.drop("TransactionDT", axis=1, inplace=True)

    print(f"Dataset merged and preprocessed! Resulted shape: {united.shape}")

    return united

In [6]:
united_dataset = preprocess_dataset(*read_dataset())

Data was loaded!
Data was loaded!
Data was loaded!
Data was loaded!
Reading train data...
Reading transactions data...
Reading identity data...
Reading test data...
Reading transactions data...
Reading identity data...
Start data preprocessing...
Dataset merged and preprocessed! Resulted shape: (500, 434)


### Train-test split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(united_dataset.drop(TARGET_COLUMN, axis=1), united_dataset[TARGET_COLUMN], test_size=0.25)

### Wrap dataset with Giskard

In [8]:
raw_dataset = pd.concat([X_test, y_test], axis=1)
wrapped_dataset = Dataset(raw_dataset,
                          name="fraud_detection_adversarial_dataset",
                          target=TARGET_COLUMN,
                          cat_columns=CATEGORICALS)

## Model training

### Build estimator

In [9]:
# Define parameters of an estimator.
ESTIMATOR_PARAMS = {
    'num_leaves': 64,
    'objective': 'binary',
    'min_data_in_leaf': 10,
    'learning_rate': 0.1,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.9,
    'bagging_freq': 1,
    'max_cat_to_onehot': 128,
    'metric': 'auc',
    'n_jobs': -1,
    'seed': 42,
    'subsample_for_bin': united_dataset.shape[0]
}

estimator = LGBMClassifier(**ESTIMATOR_PARAMS)
estimator.fit(X_train, y_train)

train_metric = roc_auc_score(y_train, estimator.predict_proba(X_train)[:, 1].T)
test_metric = roc_auc_score(y_test, estimator.predict_proba(X_test)[:, 1].T)

print(f"Train ROC-AUC score: {train_metric:.2f}")
print(f"Test ROC-AUC score: {test_metric:.2f}")

[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
Train ROC-AUC score: 1.00
Test ROC-AUC score: 0.96


### Wrap model with Giskard

In [10]:
def prediction_function(df: pd.DataFrame) -> np.ndarray:
    return estimator.predict_proba(df)

In [11]:
wrapped_model = Model(prediction_function,
                      model_type="classification",
                      name="train_test_data_classifier",
                      feature_names=X_train.columns,
                      classification_threshold=0.5,
                      classification_labels=[0, 1])

# Validate wrapped model.
wrapped_test_metric = roc_auc_score(y_test, wrapped_model.predict(wrapped_dataset).raw[:, 1].T)
print(f"Wrapped Test ROC-AUC score: {wrapped_test_metric:.2f}")

Wrapped Test ROC-AUC score: 0.96


## Scan your model to find vulnerabilities
With the Giskard scan feature, you can detect vulnerabilities in your model, including performance biases, unrobustness, data leakage, stochasticity, underconfidence, ethical issues, and more. For detailed information about the scan feature, please refer to our scan documentation.

In [12]:
results = giskard.scan(wrapped_model, wrapped_dataset)

Your model is successfully validated.
Running scan…
2023-06-02 17:48:52,103 pid:9429 MainThread giskard.scanner.logger INFO     Running detectors: ['PerformanceBiasDetector', 'TextPerturbationDetector', 'EthicalBiasDetector', 'DataLeakageDetector', 'StochasticityDetector', 'OverconfidenceDetector', 'UnderconfidenceDetector']
Running detector PerformanceBiasDetector…2023-06-02 17:48:52,104 pid:9429 MainThread giskard.scanner.logger INFO     PerformanceBiasDetector: Running
2023-06-02 17:48:52,106 pid:9429 MainThread giskard.scanner.logger INFO     PerformanceBiasDetector: Calculating loss
2023-06-02 17:48:52,382 pid:9429 MainThread giskard.scanner.logger INFO     PerformanceBiasDetector: Loss calculated (took 0:00:00.274435)
2023-06-02 17:48:52,382 pid:9429 MainThread giskard.scanner.logger INFO     PerformanceBiasDetector: Finding data slices
2023-06-02 17:49:58,723 pid:9429 MainThread giskard.scanner.logger INFO     PerformanceBiasDetector: 260 slices found (took 0:01:06.338768)
2023-

In [13]:
display(results)

<iframe id="scan-4833344864" srcdoc="
<style>pre code.hljs{display:block;overflow-x:auto;padding:1em}code.hljs{padding:3px 5px}
/*!
 Theme: GitHub Dark
 Description: Dark theme as seen on github.com
 Author: github.com
 Maintainer: @Hirse
 Updated: 2021-05-15

 Outdated base version: https://github.com/primer/github-syntax-dark
 Current colors taken from GitHub's CSS
*/.hljs{background:#0d1117;color:#c9d1d9}.hljs-doctag,.hljs-keyword,.hljs-meta .hljs-keyword,.hljs-template-tag,.hljs-template-variable,.hljs-type,.hljs-variable.language_{color:#ff7b72}.hljs-title,.hljs-title.class_,.hljs-title.class_.inherited__,.hljs-title.function_{color:#d2a8ff}.hljs-attr,.hljs-attribute,.hljs-literal,.hljs-meta,.hljs-number,.hljs-operator,.hljs-selector-attr,.hljs-selector-class,.hljs-selector-id,.hljs-variable{color:#79c0ff}.hljs-meta .hljs-string,.hljs-regexp,.hljs-string{color:#a5d6ff}.hljs-built_in,.hljs-symbol{color:#ffa657}.hljs-code,.hljs-comment,.hljs-formula{color:#8b949e}.hljs-name,.hljs-quote,.hljs-selector-pseudo,.hljs-selector-tag{color:#7ee787}.hljs-subst{color:#c9d1d9}.hljs-section{color:#1f6feb;font-weight:700}.hljs-bullet{color:#f2cc60}.hljs-emphasis{color:#c9d1d9;font-style:italic}.hljs-strong{color:#c9d1d9;font-weight:700}.hljs-addition{background-color:#033a16;color:#aff5b4}.hljs-deletion{background-color:#67060c;color:#ffdcd7}.hljs-copy-wrapper{overflow:hidden;position:relative}.hljs-copy-button:focus,.hljs-copy-wrapper:hover .hljs-copy-button{transform:translateX(0)}.hljs-copy-button{background-color:#2d2b57;background-color:var(--hljs-theme-background);background-image:url('data:image/svg+xml;charset=utf-8,<svg xmlns="http://www.w3.org/2000/svg" width="16" height="16" fill="none" viewBox="0 0 24 24"><path fill="%23fff" fill-rule="evenodd" d="M6 5a1 1 0 0 0-1 1v14a1 1 0 0 0 1 1h12a1 1 0 0 0 1-1V6a1 1 0 0 0-1-1h-2a1 1 0 1 1 0-2h2a3 3 0 0 1 3 3v14a3 3 0 0 1-3 3H6a3 3 0 0 1-3-3V6a3 3 0 0 1 3-3h2a1 1 0 0 1 0 2H6Z" clip-rule="evenodd"/><path fill="%23fff" fill-rule="evenodd" d="M7 3a2 2 0 0 1 2-2h6a2 2 0 0 1 2 2v2a2 2 0 0 1-2 2H9a2 2 0 0 1-2-2V3Zm8 0H9v2h6V3Z" clip-rule="evenodd"/></svg>');background-position:50%;background-repeat:no-repeat;border:1px solid #ffffff22;border-radius:.25rem;color:#fff;height:2rem;position:absolute;right:1em;text-indent:-9999px;top:1em;transition:background-color .2s ease,transform .2s ease-out;width:2rem}.hljs-copy-button:hover{border-color:#ffffff44}.hljs-copy-button:active{border-color:#ffffff66}.hljs-copy-button[data-copied=true]{background-image:none;text-indent:0;width:auto}@media (prefers-reduced-motion){.hljs-copy-button{transition:none}}.hljs-copy-alert{clip:rect(0 0 0 0);-webkit-clip-path:inset(50%);clip-path:inset(50%);height:1px;overflow:hidden;position:absolute;white-space:nowrap;width:1px}
/*! tailwindcss v3.3.2 | MIT License | https://tailwindcss.com*/*,:after,:before{border:0 solid #e5e7eb;box-sizing:border-box}:after,:before{--tw-content:""}html{-webkit-text-size-adjust:100%;font-feature-settings:normal;font-family:ui-sans-serif,system-ui,-apple-system,BlinkMacSystemFont,Segoe UI,Roboto,Helvetica Neue,Arial,Noto Sans,sans-serif,Apple Color Emoji,Segoe UI Emoji,Segoe UI Symbol,Noto Color Emoji;font-variation-settings:normal;line-height:1.5;-moz-tab-size:4;-o-tab-size:4;tab-size:4}body{line-height:inherit;margin:0}hr{border-top-width:1px;color:inherit;height:0}abbr:where([title]){-webkit-text-decoration:underline dotted;text-decoration:underline dotted}h1,h2,h3,h4,h5,h6{font-size:inherit;font-weight:inherit}a{color:inherit;text-decoration:inherit}b,strong{font-weight:bolder}code,kbd,pre,samp{font-family:ui-monospace,SFMono-Regular,Menlo,Monaco,Consolas,Liberation Mono,Courier New,monospace;font-size:1em}small{font-size:80%}sub,sup{font-size:75%;line-height:0;position:relative;vertical-align:baseline}sub{bottom:-.25em}sup{top:-.5em}table{border-collapse:collapse;border-color:inherit;text-indent:0}button,input,optgroup,select,textarea{color:inherit;font-family:inherit;font-size:100%

## Generate a test suite from the Scan
The objects produced by the scan can be used as fixtures to generate a test suite that integrate domain-specific issues. To create custom tests, refer to the Test your ML Model page.

In [14]:
test_suite = results.generate_test_suite("My first test suite")
test_suite.run()

Executed 'Recall on data slice “`V250` == 1.000”' with arguments {'model': <giskard.models.function.PredictionFunctionModel object at 0x11fcc7eb0>, 'dataset': <giskard.datasets.base.Dataset object at 0x11faff670>, 'slicing_function': <giskard.slicing.slice.QueryBasedSliceFunction object at 0x120071f60>, 'threshold': 0.8651785714285714}: 
               Test failed
               Metric: 0.67
               
               
Executed 'Recall on data slice “`V251` == 1.000”' with arguments {'model': <giskard.models.function.PredictionFunctionModel object at 0x11fcc7eb0>, 'dataset': <giskard.datasets.base.Dataset object at 0x11faff670>, 'slicing_function': <giskard.slicing.slice.QueryBasedSliceFunction object at 0x120010970>, 'threshold': 0.8651785714285714}: 
               Test failed
               Metric: 0.67
               
               
Executed 'Recall on data slice “`V225` == 0.000e+00”' with arguments {'model': <giskard.models.function.PredictionFunctionModel object at 0x11fcc7

(False,
 [('Recall on data slice “`V250` == 1.000”', 
                  Test failed
                  Metric: 0.67
                  
                  ),
  ('Recall on data slice “`V251` == 1.000”',
   
                  Test failed
                  Metric: 0.67
                  
                  ),
  ('Recall on data slice “`V225` == 0.000e+00”',
   
                  Test failed
                  Metric: 0.67
                  
                  ),
  ('Recall on data slice “`V269` == 0.000e+00”',
   
                  Test failed
                  Metric: 0.67
                  
                  ),
  ('Recall on data slice “`V255` == 1.000”',
   
                  Test failed
                  Metric: 0.67
                  
                  ),
  ('Recall on data slice “`V187` == 1.000”',
   
                  Test failed
                  Metric: 0.67
                  
                  ),
  ('Precision on data slice “`D10` >= 1.500 AND `D10` < 129.500”',
   
                  Test failed
                  Metric: 0.67
                  
                  ),
  ('Precision on data slice “`C13` >= 1.500 AND `C13` < 7.500”',
   
                  Test failed
                  Metric: 0.68
                  
                  ),
  ('Precision on data slice “`D5` >= 14.500”',
   
                  Test failed
                  Metric: 0.69
                  
                  ),
  ('Recall on data slice “`V199` == 1.000”',
   
                  Test failed
                  Metric: 0.75
                  
                  ),
  ('Recall on data slice “`V195` == 1.000”',
   
                  Test failed
                  Metric: 0.75
                  
                  ),
  ('Recall on data slice “`V206` == 0.000e+00”',
   
                  Test failed
                  Metric: 0.75
                  
                  ),
  ('Recall on data slice “`V302` == 1.000”',
   
                  Test failed
                  Metric: 0.75
                  
                  ),
  ('Recall on data slice “`V262` == 1.000”',
   
                  Test failed
                  Metric: 0.75
                  
                  ),
  ('Recall on data slice “`V226` == 0.000e+00”',
   
                  Test failed
                  Metric: 0.75
                  
                  )])

## Customize your suite by loading objects from the Giskard catalog

The Giskard open source catalog will enable to load:
* Tests such as metamorphic, performance, prediction & data drift, statistical tests, etc
* Slicing functions such as detectors of toxicity, hate, emotion, etc
* Transformation functions such as generators of typos, paraphrase, style tune, etc

For demo purposes, we will load a simple unit test (test_f1) that checks if the test F1 score is above the given threshold. For more examples of tests and functions, refer to the Giskard catalog.

In [15]:
test_suite.add_test(testing.test_f1(model=wrapped_model, dataset=wrapped_dataset, threshold=0.7)).run()

Executed 'Recall on data slice “`V250` == 1.000”' with arguments {'model': <giskard.models.function.PredictionFunctionModel object at 0x11fcc7eb0>, 'dataset': <giskard.datasets.base.Dataset object at 0x11faff670>, 'slicing_function': <giskard.slicing.slice.QueryBasedSliceFunction object at 0x120071f60>, 'threshold': 0.8651785714285714}: 
               Test failed
               Metric: 0.67
               
               
Executed 'Recall on data slice “`V251` == 1.000”' with arguments {'model': <giskard.models.function.PredictionFunctionModel object at 0x11fcc7eb0>, 'dataset': <giskard.datasets.base.Dataset object at 0x11faff670>, 'slicing_function': <giskard.slicing.slice.QueryBasedSliceFunction object at 0x120010970>, 'threshold': 0.8651785714285714}: 
               Test failed
               Metric: 0.67
               
               
Executed 'Recall on data slice “`V225` == 0.000e+00”' with arguments {'model': <giskard.models.function.PredictionFunctionModel object at 0x11fcc7

(False,
 [('Recall on data slice “`V250` == 1.000”', 
                  Test failed
                  Metric: 0.67
                  
                  ),
  ('Recall on data slice “`V251` == 1.000”',
   
                  Test failed
                  Metric: 0.67
                  
                  ),
  ('Recall on data slice “`V225` == 0.000e+00”',
   
                  Test failed
                  Metric: 0.67
                  
                  ),
  ('Recall on data slice “`V269` == 0.000e+00”',
   
                  Test failed
                  Metric: 0.67
                  
                  ),
  ('Recall on data slice “`V255` == 1.000”',
   
                  Test failed
                  Metric: 0.67
                  
                  ),
  ('Recall on data slice “`V187` == 1.000”',
   
                  Test failed
                  Metric: 0.67
                  
                  ),
  ('Precision on data slice “`D10` >= 1.500 AND `D10` < 129.500”',
   
                  Test failed
                  Metric: 0.67
                  
                  ),
  ('Precision on data slice “`C13` >= 1.500 AND `C13` < 7.500”',
   
                  Test failed
                  Metric: 0.68
                  
                  ),
  ('Precision on data slice “`D5` >= 14.500”',
   
                  Test failed
                  Metric: 0.69
                  
                  ),
  ('Recall on data slice “`V199` == 1.000”',
   
                  Test failed
                  Metric: 0.75
                  
                  ),
  ('Recall on data slice “`V195` == 1.000”',
   
                  Test failed
                  Metric: 0.75
                  
                  ),
  ('Recall on data slice “`V206` == 0.000e+00”',
   
                  Test failed
                  Metric: 0.75
                  
                  ),
  ('Recall on data slice “`V302` == 1.000”',
   
                  Test failed
                  Metric: 0.75
                  
                  ),
  ('Recall on data slice “`V262` == 1.000”',
   
                  Test failed
                  Metric: 0.75
                  
                  ),
  ('Recall on data slice “`V226` == 0.000e+00”',
   
                  Test failed
                  Metric: 0.75
                  
                  ),
  ('F1',
   
                  Test succeed
                  Metric: 0.89
                  
                  )])

## Upload your suite to the Giskard server

Upload your suite to the Giskard server to:
* Compare models to decide which model to promote
* Debug your tests to diagnose the issues
* Create more domain-specific tests that are integrating business feedback
* Share your results

In [ ]:
# Uploading the test suite will automatically save the model, dataset, tests, slicing & transformation functions inside the Giskard UI server
# Create a Giskard client after having install the Giskard server (see documentation)
token = "API_TOKEN"  # Find it in Settings in the Giskard server

client = GiskardClient(
    url="http://localhost:19000",  # URL of your Giskard instance
    token=token
)

my_project = client.create_project("my_project", "PROJECT_NAME", "DESCRIPTION")

# Upload to the current project ✉️
test_suite.upload(client, "my_project")